In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3) -> None:
        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x

In [ ]:
torch.manual_seed(32)
model = Model()

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('data/iris.csv')
df.head()

In [ ]:
targets = df['variety'].values
encoded = []
for i in targets:
    if(i == 'Setosa'):
        encoded.append(0.0)
    elif(i == 'Versicolor'):
        encoded.append(1.0)
    else:
        encoded.append(2.0)
df['variety'] = encoded
df.head()

In [ ]:
X = df.drop('variety', axis=1)
y = df['variety']

In [ ]:
X = X.values
y = y.values
print(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [ ]:
print(y)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
print(model.parameters)

In [ ]:
epochs = 100
losses = []

for i in range(epochs):
    
    optimizer.zero_grad()

    y_pred = model.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss.item())
    
    if i%10==0:
        print(f"epoch {i} loss: {loss.item()}")
    
    loss.backward()
    optimizer.step()

In [ ]:
plt.plot(range(epochs), losses)
plt.ylabel('LOSS')
plt.xlabel('Epoch')

In [ ]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss = criterion(y_eval, y_test)

print(loss)

In [ ]:
correct = 0

with torch.no_grad():
    
    for i,data in enumerate(X_test):
        y_val = model.forward(data)

        print(f'{i+1}.) {str(y_val)} {y_test[i]}')

        if(y_val.argmax().item() == y_test[i]):
            correct += 1

print(f'Success ratio: {correct}/{y_test.shape[0]}')

In [ ]:
torch.save(model, 'my_iris_model.pt')

In [ ]:
new_model = Model()
#new_model.load_state_dict(torch.load('my_iris_model.pt'))
new_model = torch.load('my_iris_model.pt')

In [ ]:
new_model.eval()

In [ ]:
mystery_iris = torch.tensor([5.6, 3.7, 2.2, 0.5])

with torch.no_grad():
    print(new_model(mystery_iris))

    